In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window as W

In [2]:
spark = (SparkSession.builder
        .master("local")
        .appName("PysparkSQL_RJ")
        .config("spark.ui.port", "4050")
        .getOrCreate()
        )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [4]:
df = (spark.read.format("parquet")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("gs://projeto_final_2021/parquet_to_bq/RJ/part-00000-9403edb2-d06c-4fc2-aa5f-555a889b8f6f-c000.snappy.parquet")
  .createOrReplaceTempView("rj")
     )

In [5]:
# Descrição de dados da tabela 
spark.sql('DESCRIBE RJ').show(30)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
|           mes|  tinyint|   null|
|           ano| smallint|   null|
|          nome|   string|   null|
|         cargo|   string|   null|
|      situacao|   string|   null|
|salarioLiquido|    float|   null|
|        estado|   string|   null|
+--------------+---------+-------+



In [6]:
# Consulta da tabela inteira 
spark.sql('''
            SELECT
                *
            FROM
                rj
              
          ''').show(30)

+---+----+--------------------+--------------------+--------+--------------+------+
|mes| ano|                nome|               cargo|situacao|salarioLiquido|estado|
+---+----+--------------------+--------------------+--------+--------------+------+
|  8|2021|ABEIGAIR DE ALENC...|    ENFERMEIRO - ESF|   ATIVO|       4281.61|    RJ|
|  8|2021|ADAIL JULIO DO NA...|    GUARDA MUNICIPAL|   ATIVO|       4013.84|    RJ|
|  8|2021|ADALBERTO CURVELO...|SUBSECRETÁRIO DE ...|   ATIVO|       3799.06|    RJ|
|  8|2021|ADEILTON ALVES DE...|    GUARDA MUNICIPAL|   ATIVO|       1220.18|    RJ|
|  8|2021|ADEMILTON DE OLIV...|  TRABALHADOR BRAÇAL|   ATIVO|       2226.98|    RJ|
|  8|2021|ADERLANE  CRISTIN...|TECNICO EM ENFERM...|   ATIVO|       1372.31|    RJ|
|  8|2021|ADILSON MACHADO M...|AUXILIAR DE ARTIFICE|   ATIVO|       2511.15|    RJ|
|  8|2021|ADJANE BASILIO FA...|         PROFESSOR I|   ATIVO|        182.93|    RJ|
|  8|2021|ADRIANA APARECIDA...|ASSESSOR DE INTEG...|   ATIVO|       1973.87|

In [20]:
#Quantidade de servidores por cargo

spark.sql('''SELECT cargo, COUNT(nome) AS Qtd_servidores FROM rj GROUP BY cargo''').show()

+--------------------+--------------+
|               cargo|Qtd_servidores|
+--------------------+--------------+
|  TRABALHADOR BRAÇAL|          3703|
|CHEFE DA SEÇÃO DE...|             4|
|ASSESSOR FINANCEI...|            23|
|CHEFE DA SEÇÃO DE...|             3|
|ARTIFICE DE MECAN...|             8|
|AUXILIAR DE SERVI...|           889|
|ASSESSOR DE INTEG...|            90|
|TECNICO EM ENFERM...|           227|
|CHEFE DA SEÇÃO DE...|             4|
|AUXILIAR DE PLANE...|            18|
|DIRETOR DE UNIDAD...|             3|
|AUXILIAR OPERACIO...|            54|
|DIRETOR  ? E.M. P...|             3|
|COORDENADOR DE AR...|            69|
|DIRETOR ? C. M. E...|             3|
|SECRETÁRIO MUNICI...|            12|
|ASSESSOR DE INTEG...|            10|
|SECRETÁRIO MUNICI...|            22|
|CHEFE DA SEÇÃO DE...|             3|
|TECNICO EM EDIFIC...|            42|
+--------------------+--------------+
only showing top 20 rows



In [7]:
# Verificar ano, cargo e salario maior que 10000 no ano de 2021
spark.sql("""
SELECT
    ano, cargo, salarioLiquido 
FROM 
    rj
WHERE
    salarioLiquido > 10000 AND ano = 2021

""").show(30)


+----+--------------------+--------------+
| ano|               cargo|salarioLiquido|
+----+--------------------+--------------+
|2021|SECRETÁRIO MUN EX...|      12135.13|
|2021|  PREFEITO MUNICIPAL|      19877.41|
|2021|SECRETÁRIO MUN EX...|      12135.13|
|2021|  PREFEITO MUNICIPAL|      19877.41|
|2021|SECRETÁRIO MUN EX...|      12135.13|
|2021|  PREFEITO MUNICIPAL|      19877.41|
|2021|  PREFEITO MUNICIPAL|      10787.98|
|2021|SECRETÁRIO DE GOV...|      12135.13|
|2021|  PREFEITO MUNICIPAL|      19877.41|
|2021|SECRETÁRIO DE GOV...|      12135.13|
|2021|  PREFEITO MUNICIPAL|      19877.41|
|2021|SECRETÁRIO DE GOV...|      12135.13|
|2021|  PREFEITO MUNICIPAL|      19877.41|
|2021|  PREFEITO MUNICIPAL|      19877.41|
|2021|  PREFEITO MUNICIPAL|      19877.41|
+----+--------------------+--------------+



In [34]:
#Média salarial dos servidores no cargo de professor durante o decorrer do ano de 2020
spark.sql(
"""
SELECT
    cargo,
    ROUND(AVG(salarioLiquido),2) AS media_salario,
    ano,
    mes 
FROM
    rj
WHERE
    cargo LIKE "%PROF%" AND ano == 2020
GROUP BY
    mes, ano, cargo
ORDER BY
    mes
"""
).show(truncate=50)

+-------------------------------------------+-------------+----+---+
|                                      cargo|media_salario| ano|mes|
+-------------------------------------------+-------------+----+---+
|           PROFESSOR II - LINGUA PORTUGUESA|      2408.45|2020|  1|
|                    PROFESSOR II - HISTORIA|      1999.96|2020|  1|
|          PROFESSOR II - EDUCAÇÃO ARTISTICA|      3383.69|2020|  1|
|                  PROFESSOR II - MATEMATICA|      2721.87|2020|  1|
|                                PROFESSOR I|      1858.86|2020|  1|
|                      PROFESSOR II - INGLES|       2583.0|2020|  1|
|                   PROFESSOR II - GEOGRAFIA|      2422.21|2020|  1|
|PROFESSOR DE INFORMAÇÃO E EDUCAÇÃO EM SAÚDE|      2759.84|2020|  1|
|             PROFESSOR II - EDUCAÇÃO FÍSICA|      2508.43|2020|  1|
|                    PROFESSOR II - CIENCIAS|      2295.55|2020|  1|
|                 PROFESSOR INSPETOR ESCOLAR|      2523.96|2020|  1|
|                    PROFESSOR II 

In [23]:
#Top 5 menores médias salariais por cargo 
spark.sql("SELECT cargo, ROUND(AVG(salarioLiquido),2) AS media_salario_cargo FROM rj GROUP BY cargo ORDER BY media_salario_cargo limit 5").show()

+--------------------+-------------------+
|               cargo|media_salario_cargo|
+--------------------+-------------------+
|CHEFE DA SEÇÃO DE...|              41.87|
|DIRETOR ADJUNTO ?...|             141.79|
|CHEFE DA SEÇÃO DE...|             261.85|
|CHEFE DA SEÇÃO DE...|             663.92|
|          DESENHISTA|             666.08|
+--------------------+-------------------+



In [35]:
# Verificar a quantidade de pagamento efetuado para cada servidor 
spark.sql('''
            SELECT 
                nome,
            COUNT
                (salarioLiquido) 
            FROM 
                rj 
            GROUP BY 
                nome 
            LIMIT 10
            ''').show()

+--------------------+---------------------+
|                nome|count(salarioLiquido)|
+--------------------+---------------------+
| ALAN ROGERIO BALDEZ|                   23|
|ANDREZA RAFAELA F...|                   24|
|EDER DO AMARAL MA...|                   20|
|ELTON JOSE SALGAD...|                   23|
| FAGNER SOARES DIOGO|                   23|
|FERNANDO JOSE PER...|                   23|
|JAQUELINE FRAGA D...|                   23|
|LILIANE MOREIRA B...|                   25|
|MARIA SOUZA DA SILVA|                   23|
|RAQUEL CRISNER DO...|                   23|
+--------------------+---------------------+



In [9]:
spark.sql("SELECT DISTINCT(COUNT(nome)) AS nome_diferentes FROM rj").show() 

+---------------+
|nome_diferentes|
+---------------+
|          32333|
+---------------+



In [24]:
#Descobrir o número total de registros de cada ano
spark.sql("select ano, count(situacao) as total_pagamentos from rj group by ano").show() 

+----+---------------+
| ano|count(situacao)|
+----+---------------+
|2020|          19887|
|2021|          12446|
+----+---------------+



In [10]:
spark.sql("SELECT COUNT(nome) AS nome FROM rj").show() 


+-----+
| nome|
+-----+
|32333|
+-----+



In [11]:
# Verificar nome, cargo e situção, no mês de 4 de 2020
spark.sql("SELECT nome,cargo,situacao FROM rj  where mes = 4 and ano = 2020").show()

+--------------------+--------------------+--------+
|                nome|               cargo|situacao|
+--------------------+--------------------+--------+
|ABEIGAIR DE ALENC...|    ENFERMEIRO - ESF|   ATIVO|
|ACACIA PEREIRA DA...|COORDENADOR DE CE...|   ATIVO|
|ADAIL JULIO DO NA...|    GUARDA MUNICIPAL|   ATIVO|
|ADALBERTO CURVELO...|SECRETÁRIO MUNICI...|   ATIVO|
|ADEILTON ALVES DE...|    GUARDA MUNICIPAL|   ATIVO|
|ADEMILTON DE OLIV...|  TRABALHADOR BRAÇAL|   ATIVO|
|ADENIRA DA SILVEI...|ASSISTENTE OPERAC...|   ATIVO|
|ADILSON MACHADO M...|AUXILIAR DE ARTIFICE|   ATIVO|
|ADJANE BASILIO FA...|         PROFESSOR I|   ATIVO|
|ADRIANA APARECIDA...|AUXILIAR OPERACIO...|   ATIVO|
|ADRIANA CHAGAS DO...|AUXILIAR DE ENFER...|   ATIVO|
|ADRIANA DA CONCEI...|AUXILIAR ADMINIST...|   ATIVO|
|ADRIANA DE CARVAL...|         PROFESSOR I|   ATIVO|
|ADRIANA DE JESUS ...|          MERENDEIRA|   ATIVO|
|ADRIANA DRUMMOND ...|MEDICO VETERINARI...|   ATIVO|
|ADRIANA FERREIRA ...|AUXILIAR DE SERVI...|   

In [12]:
# Verificar os cargos existentes
spark.sql(
'''
SELECT
    cargo
FROM
    rj
GROUP By
cargo

'''

).show()

+--------------------+
|               cargo|
+--------------------+
|  TRABALHADOR BRAÇAL|
|CHEFE DA SEÇÃO DE...|
|ASSESSOR FINANCEI...|
|CHEFE DA SEÇÃO DE...|
|ARTIFICE DE MECAN...|
|AUXILIAR DE SERVI...|
|ASSESSOR DE INTEG...|
|TECNICO EM ENFERM...|
|CHEFE DA SEÇÃO DE...|
|AUXILIAR DE PLANE...|
|DIRETOR DE UNIDAD...|
|AUXILIAR OPERACIO...|
|DIRETOR  ? E.M. P...|
|COORDENADOR DE AR...|
|DIRETOR ? C. M. E...|
|SECRETÁRIO MUNICI...|
|ASSESSOR DE INTEG...|
|SECRETÁRIO MUNICI...|
|CHEFE DA SEÇÃO DE...|
|TECNICO EM EDIFIC...|
+--------------------+
only showing top 20 rows



In [25]:
#Quantidade de servidores por cargo

spark.sql('''SELECT cargo, COUNT(nome) AS Qtd_servidores FROM rj GROUP BY cargo''').show()

+--------------------+--------------+
|               cargo|Qtd_servidores|
+--------------------+--------------+
|  TRABALHADOR BRAÇAL|          3703|
|CHEFE DA SEÇÃO DE...|             4|
|ASSESSOR FINANCEI...|            23|
|CHEFE DA SEÇÃO DE...|             3|
|ARTIFICE DE MECAN...|             8|
|AUXILIAR DE SERVI...|           889|
|ASSESSOR DE INTEG...|            90|
|TECNICO EM ENFERM...|           227|
|CHEFE DA SEÇÃO DE...|             4|
|AUXILIAR DE PLANE...|            18|
|DIRETOR DE UNIDAD...|             3|
|AUXILIAR OPERACIO...|            54|
|DIRETOR  ? E.M. P...|             3|
|COORDENADOR DE AR...|            69|
|DIRETOR ? C. M. E...|             3|
|SECRETÁRIO MUNICI...|            12|
|ASSESSOR DE INTEG...|            10|
|SECRETÁRIO MUNICI...|            22|
|CHEFE DA SEÇÃO DE...|             3|
|TECNICO EM EDIFIC...|            42|
+--------------------+--------------+
only showing top 20 rows



In [14]:
# Verificar cargos distintos
spark.sql(
'''
SELECT 
    DISTINCT(cargo)
FROM
    rj
'''
).show(100)

+--------------------+
|               cargo|
+--------------------+
|  TRABALHADOR BRAÇAL|
|CHEFE DA SEÇÃO DE...|
|ASSESSOR FINANCEI...|
|CHEFE DA SEÇÃO DE...|
|ARTIFICE DE MECAN...|
|AUXILIAR DE SERVI...|
|ASSESSOR DE INTEG...|
|TECNICO EM ENFERM...|
|CHEFE DA SEÇÃO DE...|
|AUXILIAR DE PLANE...|
|DIRETOR DE UNIDAD...|
|AUXILIAR OPERACIO...|
|DIRETOR  ? E.M. P...|
|COORDENADOR DE AR...|
|DIRETOR ? C. M. E...|
|SECRETÁRIO MUNICI...|
|ASSESSOR DE INTEG...|
|SECRETÁRIO MUNICI...|
|CHEFE DA SEÇÃO DE...|
|TECNICO EM EDIFIC...|
|CHEFE DA DIVISÃO ...|
|   ARTIFICE MECANICA|
|ASSESSOR OPERACIO...|
|DIRETOR ? E. E. M...|
|ASSISTENTE DE MAN...|
|TECNICO EM ENFERM...|
|ASSESSOR ADMINIST...|
|PROFESSOR II - ED...|
|SUBSECRETÁRIO DE ...|
|AUXILIAR DE DESEN...|
|COORDENADOR DO CR...|
|SUBSECRETÁRIO MUN...|
|COORDENADOR DA PR...|
| AUXILIAR DE FAZENDA|
|PROFESSOR II - LÍ...|
|CHEFE DA DIVISÃO ...|
|SUBSECRETÁRIO DE ...|
|PROFESSOR I - CON...|
|COORDENADOR DO FU...|
|AGENTE DE SAUDE P...|
|COORDENADO

In [15]:
# média salarial por estado dos servidores ativos 
spark.sql(
'''
SELECT 
    ROUND(AVG(salarioLiquido),2) AS media_salarial ,
    estado
FROM 
    rj
WHERE
    situacao = "ATIVO"
GROUP BY
    estado
'''
).show()


+--------------+------+
|media_salarial|estado|
+--------------+------+
|       1975.01|    RJ|
+--------------+------+



In [16]:
#verificar dados no mês de fevereiro de 2021
spark.sql("SELECT * FROM rj WHERE mes = 2 AND ano = 2021").show()

+---+----+--------------------+--------------------+--------+--------------+------+
|mes| ano|                nome|               cargo|situacao|salarioLiquido|estado|
+---+----+--------------------+--------------------+--------+--------------+------+
|  2|2021|ABEIGAIR DE ALENC...|    ENFERMEIRO - ESF|   ATIVO|       5672.84|    RJ|
|  2|2021|ADAIL JULIO DO NA...|    GUARDA MUNICIPAL|   ATIVO|        7751.5|    RJ|
|  2|2021|ADALBERTO CURVELO...|SUBSECRETÁRIO DE ...|   ATIVO|       3799.06|    RJ|
|  2|2021|ADEILTON ALVES DE...|    GUARDA MUNICIPAL|   ATIVO|       1980.26|    RJ|
|  2|2021|ADEMILTON DE OLIV...|  TRABALHADOR BRAÇAL|   ATIVO|       1986.65|    RJ|
|  2|2021|ADILSON MACHADO M...|AUXILIAR DE ARTIFICE|   ATIVO|       3046.69|    RJ|
|  2|2021|ADJANE BASILIO FA...|         PROFESSOR I|   ATIVO|       1172.24|    RJ|
|  2|2021|ADRIANA APARECIDA...|AUXILIAR OPERACIO...|   ATIVO|        1696.2|    RJ|
|  2|2021|ADRIANA CHAGAS DO...|AUXILIAR DE ENFER...|   ATIVO|       1772.16|

In [17]:
#Valores gastos com o sálario ao longo dos mês ao longo do ano de 2021
spark.sql(
'''
SELECT
    mes,
    ROUND(Sum(salarioLiquido),2) AS soma_salario
FROM
    rj
WHERE 
    ano = 2021
GROUP BY 
    mes
ORDER BY
    mes ASC
 
''').show()

+---+------------+
|mes|soma_salario|
+---+------------+
|  1|  2748145.54|
|  2|  2853094.06|
|  3|  2856688.25|
|  4|  2934039.26|
|  5|  2955164.16|
|  6|  4600505.18|
|  7|  3032639.45|
|  8|  3067498.47|
+---+------------+



In [33]:
# Valores gastos com salário ao longo dos anos
spark.sql('''
SELECT
    ano,
    ROUND(Sum(salarioLiquido),2) as soma_salario
FROM 
    rj
GROUP BY
    ano
''').show()

+----+-------------+
| ano| soma_salario|
+----+-------------+
|2020|3.881027531E7|
|2021|2.504777438E7|
+----+-------------+



In [27]:
# Média salarial dos servidores ativos no ano de 2021
spark.sql(
"""
SELECT
    cargo,
    ROUND(AVG(salarioLiquido),2) AS media_salario,
    ano,
    mes,
    situacao
FROM
    rj
WHERE
    situacao = "ATIVO" AND ano = 2021
GROUP BY
    mes, ano, cargo, situacao
ORDER BY
    mes
"""
).show()

+--------------------+-------------+----+---+--------+
|               cargo|media_salario| ano|mes|situacao|
+--------------------+-------------+----+---+--------+
|  PROCURADOR ADJUNTO|      3227.36|2021|  1|   ATIVO|
|FISCAL DE OBRAS E...|      1307.47|2021|  1|   ATIVO|
|TECNICO EM RADIOL...|       4907.8|2021|  1|   ATIVO|
|DIRETOR DE OBRAS ...|      3906.15|2021|  1|   ATIVO|
|  CIRURGIÃO DENTISTA|      2953.41|2021|  1|   ATIVO|
|     CUIDADOR SOCIAL|       606.58|2021|  1|   ATIVO|
|SUBSECRETÁRIO DE ...|      3841.71|2021|  1|   ATIVO|
|SUBSECRETÁRIO DE ...|      3799.06|2021|  1|   ATIVO|
| ASSISTENTE CONTÁBIL|      2423.63|2021|  1|   ATIVO|
|COORDENADOR DO CR...|       2075.1|2021|  1|   ATIVO|
|SECRETÁRIO MUN DE...|      7098.52|2021|  1|   ATIVO|
|OPERADOR DE INFOR...|      1586.65|2021|  1|   ATIVO|
|SECRETÁRIO MUNICI...|      7150.65|2021|  1|   ATIVO|
|TECNICO EM EDIFIC...|       2168.4|2021|  1|   ATIVO|
|SECRETÁRIO MUNICI...|      7046.38|2021|  1|   ATIVO|
|PROFESSOR